In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('2019-Nov.csv')

In [ ]:
df= data.sample(100000)

In [1]:
import pandas as pd
data= pd.read_csv('2019-Nov.csv')

In [3]:
import threading
import time
from queue import Queue
import pandas as pd
#print class message to override defualt print message with prefix 
"""
This class represents an individual action from a user.
"""
class Message:
    def __init__(self, timestamp: int, userId: str, type: str, product_Id: str, content: dict):
        self.timestamp = timestamp
        self.userId = userId
        self.type = type
        self.product_Id= product_Id
        self.content = content

    def __str__(self):
        return f"Message:\n\tUser Id: {self.userId}\n\tAt Timestamp: {self.timestamp}\n\tType: {self.type}\n\tproduct Id: {self.product_Id}\n\tContent: {self.content}"


class KafkaBroker:
    def __init__(self):
        self.message_queues = {}

    def produce_message(self, message: Message, topic: str) -> None:
        if topic not in self.message_queues:
            self.message_queues[topic] = Queue()
        self.message_queues[topic].put(message)

    def consume_messages(self):
        while True:
            messages_exist = False
            for index, message_queue in self.message_queues.items():
                if not message_queue.empty():
                    messages_exist = True
                    yield message_queue.get()
            if not messages_exist:
                time.sleep(0.01)
                continue
            yield None  # Yield None when there are no more messages to consume

class KafkaProducer:
    def __init__(self, broker: KafkaBroker, topic: str):
        self.broker = broker
        self.topic = topic
        self.thread = None

    def start(self):
        self.thread = threading.Thread(target=self.produce_messages)
        self.thread.start()

    def send(self, message: Message):
        self.broker.produce_message(message, self.topic)

    def produce_messages(self):
        df = data.sample(n=100)

        for _, row in df.iterrows():
            timestamp = int(pd.Timestamp(row['event_time']).timestamp())
            user_id = str(row['user_id'])
            event_type = str(row['event_type'])
            product_id= row['product_id']
            content = {
                'price': row['price']
            }
            message = Message(timestamp, user_id, event_type, product_id, content)
            self.send(message)
            time.sleep(0.001)
        time.sleep(1)  # Additional delay to ensure all messages are produced

class KafkaConsumer:
    def __init__(self, broker: KafkaBroker, topic: str):
        self.broker = broker
        self.topic = topic
        self.queue = Queue()
        self.thread = None

    def start(self):
        self.thread = threading.Thread(target=self.run)
        self.thread.start()

    def run(self):
        for message in self.broker.consume_messages():
            if message is not None:
                self.queue.put(message)

    def poll(self, timeout=1):
        try:
            return self.queue.get(timeout=timeout)
        except:
            return None

if __name__ == "__main__":
    broker = KafkaBroker()
    consumer = KafkaConsumer(broker, 'user_activities')
    producer = KafkaProducer(broker, 'user_activities')
    producer.start()
    consumer.start()

    # Arbitrary timeout for testing purposes, in production, these systems would constantly be active
    timeout_seconds = 20

    # Simple counter to track the number of consumed messages
    consumed_messages = 0

    start_time = time.time()
    while True:
        message = consumer.poll()
        if message:
            consumed_messages += 1
            print(f"[#{consumed_messages}]{message}")
        elif time.time() - start_time >= timeout_seconds:
            break
        time.sleep(0.01)

    if consumed_messages > 0:
        print(f"Finished! Consumed a total of {consumed_messages} messages")
    else:
        print(f"Hmpf, no messages were consumed, maybe your implementation is wrong?")



[#1]Message:
	User Id: 557818378
	At Timestamp: 1573767385
	Type: view
	product Id: 6000227
	Content: {'price': 157.53}
[#2]Message:
	User Id: 518805065
	At Timestamp: 1573305072
	Type: view
	product Id: 17800521
	Content: {'price': 54.03}
[#3]Message:
	User Id: 543126131
	At Timestamp: 1574010673
	Type: view
	product Id: 4600551
	Content: {'price': 282.61}
[#4]Message:
	User Id: 524079631
	At Timestamp: 1574592399
	Type: view
	product Id: 1005259
	Content: {'price': 463.31}
[#5]Message:
	User Id: 569804991
	At Timestamp: 1573405812
	Type: view
	product Id: 1005197
	Content: {'price': 102.94}
[#6]Message:
	User Id: 548904793
	At Timestamp: 1573826490
	Type: view
	product Id: 26300086
	Content: {'price': 241.71}
[#7]Message:
	User Id: 566522251
	At Timestamp: 1574524445
	Type: view
	product Id: 1480106
	Content: {'price': 594.58}
[#8]Message:
	User Id: 548798420
	At Timestamp: 1574704355
	Type: view
	product Id: 100021237
	Content: {'price': 51.46}
[#9]Message:
	User Id: 514197336
	At T

In [ ]:
if __name__ == "__main__":
    broker = KafkaBroker()
    consumer = KafkaConsumer(broker, 'user_activities')
    producer = KafkaProducer(broker, 'user_activities')
    producer.start()
    consumer.start()

    # Wait for messages to be produced
    time.sleep(1)

    # Simple counter to track the number of consumed messages
    consumed_messages = 0
    timeout_seconds = 10

    start_time = time.time()
    while True:
        message = consumer.poll(timeout=1)
        if message:
            consumed_messages += 1
            print(f"[#{consumed_messages}] {message}")
        elif time.time() - start_time >= timeout_seconds:
            break

    consumer.queue.put(None)  # Signal the consumer thread to stop

    # Wait for the consumer thread to finish
    consumer.thread.join()

    if consumed_messages > 0:
        print(f"Finished! Consumed a total of {consumed_messages} messages")
    else:
        print(f"No messages were consumed. Check your implementation.")

